In [1]:
'''
Preprocess script for SNPs, mostly covering the translation part
synonomous vs non-synonomous
'''
import pandas as pd, numpy as np, re, sys, json
from pathlib import Path
from multiprocessing import Pool
from collections import Counter

in_path = Path('/d/data/plasmo/nat_out/v4')
in_file = in_path / 'snps_by_genes.tsv'
out_path = in_path / 'gene_scores.tsv'
genes_file = in_path / 'full_genes.txt'
ref_file = in_path / '3d7.fasta'
sift_file = in_path / 'sift.tsv'

In [2]:
#loading the actual SNPs
in_df = pd.read_csv(in_file, sep='\t', index_col = [0,1])
sift_df = pd.read_csv(sift_file, sep='\t', index_col = [0,1])

In [3]:
by_genes = in_df.groupby('CHROM') #each iteration yields (name, df)

In [4]:
#function for assigning score to each element of each row
def toScore(row):
    #the stipulation here is that the 'ref' are the 5 reference bases surrounding the SNP (+2, -2)
    #scoring scheme: regular = 1, stop codon or indel = 3.
    def getSingleScore(snp, ref, alts, sift_info):       
        
#         print('calling on ', snp, ref, alts)
        #ref may be a list
        if snp in ref:
            return 0 #no mutation
        
        if snp not in alts:
            return 0 #no annotation
        
#         print('scoring ', snp, ref, alts, sift_info)
        #actual scoring
        row = sift_info.loc[snp]
        
        if len(row.shape) > 1:
            row = row.head(1)
        
        if re.search('DELETERIOUS', str(row['SIFT_PREDICTION'])):
            return 1
        
#         if re.search('TOLERATED', str(row['SIFT_PREDICTION'])):
#             return 0
        
        if re.search('^FRAMESHIFT', str(row['VARIANT_TYPE'])):
            return 3
        
        if re.search('STOP', str(row['VARIANT_TYPE'])):
            return 2
        
#         if re.search('SUBSTITUTION', str(row['VARIANT_TYPE'])):
#             return 0
        
#         if re.search('NONSYNONYMOUS', str(row['VARIANT_TYPE'])):
#             return 0
        
        return 0
    
    def getSingleScoreWrapper(snp, ref, alts, sift_info):

        score = getSingleScore(snp, ref, alts, sift_info)

        return score
    
    def zeroScore(x):
        return 0
    
    global sift_df
       
    #the name of the chromosome should be id:chromosome
    full_id = row.name[0].split(':')
    id = full_id[0]
    chr = full_id[1]
    pos = int(row.name[1])
    
    try:
        #have annotation
        print('annotation at', chr, pos)
        sift_info = sift_df.loc[(chr, pos)]
    except:
        #no annotation
        print('no annotation at ', chr, pos)
        return row.apply(zeroScore)
    
    
    ref = sift_info['REF_ALLELE'].values
    alts = sift_info['ALT_ALLELE'].values
    sift_info.set_index('ALT_ALLELE', inplace = True)
    
    scores = row.apply(getSingleScoreWrapper, 1, args = (ref, alts, sift_info))

    return scores

def worker(args):
    #hard coded args
    name = args[0]
    snps_df = args[1]
    
    scores = snps_df.apply(toScore, 1)
    result = scores.apply(sum, 0)
    result = pd.DataFrame(result, columns = [name]).T
    return result

with Pool(6) as pool:
    res_dfs = pool.map(worker, by_genes)
res = pd.concat(res_dfs)

annotation at Pf3D7_01_v3 95231
annotation at Pf3D7_01_v3 98866
annotation at Pf3D7_01_v3 98978
annotation at Pf3D7_01_v3 99226
annotation at Pf3D7_01_v3 99273
annotation at Pf3D7_01_v3 100315
annotation at Pf3D7_01_v3 100325
annotation at Pf3D7_01_v3 100333
annotation at Pf3D7_01_v3 100681
annotation at Pf3D7_01_v3 100916
annotation at Pf3D7_01_v3 101208
annotation at Pf3D7_01_v3 101767
annotation at Pf3D7_01_v3 102261
annotation at Pf3D7_01_v3 104717
annotation at Pf3D7_01_v3 107519
annotation at Pf3D7_01_v3 107626
annotation at Pf3D7_01_v3 107675
annotation at Pf3D7_01_v3 107823
annotation at Pf3D7_01_v3 108002
annotation at Pf3D7_01_v3 110840
no annotation at  Pf3D7_01_v3 110840
annotation at Pf3D7_01_v3 111040
annotation at Pf3D7_01_v3 111041
annotation at Pf3D7_01_v3 111158
no annotation at  Pf3D7_01_v3 111158
annotation at Pf3D7_01_v3 111350
annotation at Pf3D7_01_v3 111867
annotation at Pf3D7_01_v3 112038
annotation at Pf3D7_01_v3 112188
annotation at Pf3D7_01_v3 112653
annotat

annotation at Pf3D7_01_v3 180379
annotation at Pf3D7_01_v3 180554
annotation at Pf3D7_01_v3 182927
annotation at Pf3D7_01_v3 183260
annotation at Pf3D7_01_v3 184114
annotation at Pf3D7_01_v3 190739
annotation at Pf3D7_01_v3 190777
annotation at Pf3D7_01_v3 191917
annotation at Pf3D7_01_v3 191947
annotation at Pf3D7_01_v3 192032
annotation at Pf3D7_01_v3 192264
annotation at Pf3D7_01_v3 192479
annotation at Pf3D7_01_v3 192498
annotation at Pf3D7_01_v3 192625
annotation at Pf3D7_01_v3 193245
annotation at Pf3D7_01_v3 193299
annotation at Pf3D7_01_v3 193365
annotation at Pf3D7_01_v3 193536
annotation at Pf3D7_01_v3 193559
annotation at Pf3D7_01_v3 193576
annotation at Pf3D7_01_v3 193667
annotation at Pf3D7_01_v3 193846
annotation at Pf3D7_01_v3 193926
annotation at Pf3D7_01_v3 195031
annotation at Pf3D7_01_v3 196099
annotation at Pf3D7_01_v3 196136
annotation at Pf3D7_01_v3 196144
annotation at Pf3D7_01_v3 196680
annotation at Pf3D7_01_v3 196974
annotation at Pf3D7_01_v3 196976
annotation

annotation at Pf3D7_01_v3 340644
annotation at Pf3D7_01_v3 341495
annotation at Pf3D7_01_v3 341609
annotation at Pf3D7_01_v3 343769
annotation at Pf3D7_01_v3 344792
no annotation at  Pf3D7_01_v3 344792
annotation at Pf3D7_01_v3 345142
annotation at Pf3D7_01_v3 345143
annotation at Pf3D7_01_v3 345583
no annotation at  Pf3D7_01_v3 345583
annotation at Pf3D7_01_v3 345670
no annotation at  Pf3D7_01_v3 345670
annotation at Pf3D7_01_v3 345730
annotation at Pf3D7_01_v3 346385
no annotation at  Pf3D7_01_v3 346385
annotation at Pf3D7_01_v3 346485
no annotation at  Pf3D7_01_v3 346485
annotation at Pf3D7_01_v3 344792
no annotation at  Pf3D7_01_v3 344792
annotation at Pf3D7_01_v3 345142
annotation at Pf3D7_01_v3 345143
annotation at Pf3D7_01_v3 345583
no annotation at  Pf3D7_01_v3 345583
annotation at Pf3D7_01_v3 345670
no annotation at  Pf3D7_01_v3 345670
annotation at Pf3D7_01_v3 345730
annotation at Pf3D7_01_v3 346385
no annotation at  Pf3D7_01_v3 346385
annotation at Pf3D7_01_v3 346485
no anno

annotation at Pf3D7_01_v3 501008
annotation at Pf3D7_01_v3 501009
annotation at Pf3D7_01_v3 501176
annotation at Pf3D7_01_v3 501360
annotation at Pf3D7_01_v3 501421
annotation at Pf3D7_01_v3 501515
annotation at Pf3D7_01_v3 502218
annotation at Pf3D7_01_v3 502412
annotation at Pf3D7_01_v3 502591
no annotation at  Pf3D7_01_v3 502591
annotation at Pf3D7_01_v3 502662
annotation at Pf3D7_01_v3 505208
annotation at Pf3D7_01_v3 505527
annotation at Pf3D7_01_v3 505617
annotation at Pf3D7_01_v3 505653
annotation at Pf3D7_01_v3 505721
annotation at Pf3D7_01_v3 505912
annotation at Pf3D7_01_v3 505927
annotation at Pf3D7_01_v3 506069
annotation at Pf3D7_01_v3 506075
annotation at Pf3D7_01_v3 506140
annotation at Pf3D7_01_v3 506772
annotation at Pf3D7_01_v3 506774
annotation at Pf3D7_01_v3 508618
annotation at Pf3D7_01_v3 508654
annotation at Pf3D7_01_v3 508725
annotation at Pf3D7_01_v3 509038
annotation at Pf3D7_01_v3 509083
annotation at Pf3D7_01_v3 509112
annotation at Pf3D7_01_v3 509256
annota

no annotation at  Pf3D7_02_v3 489992
annotation at Pf3D7_02_v3 493313
annotation at Pf3D7_02_v3 496476
annotation at Pf3D7_01_v3 530459
annotation at Pf3D7_02_v3 497012
annotation at Pf3D7_01_v3 530484
annotation at Pf3D7_02_v3 497013
annotation at Pf3D7_02_v3 498059
annotation at Pf3D7_02_v3 499889
annotation at Pf3D7_01_v3 530497
annotation at Pf3D7_01_v3 530499
annotation at Pf3D7_02_v3 500609
annotation at Pf3D7_02_v3 501282
annotation at Pf3D7_01_v3 530519
annotation at Pf3D7_02_v3 501415
annotation at Pf3D7_01_v3 530521
annotation at Pf3D7_02_v3 501530
annotation at Pf3D7_01_v3 530524
annotation at Pf3D7_01_v3 530529
annotation at Pf3D7_02_v3 501972
annotation at Pf3D7_01_v3 530536
annotation at Pf3D7_02_v3 502703
annotation at Pf3D7_02_v3 502708
annotation at Pf3D7_01_v3 530537
annotation at Pf3D7_02_v3 503169
annotation at Pf3D7_02_v3 503292
annotation at Pf3D7_01_v3 530539
annotation at Pf3D7_01_v3 530545
annotation at Pf3D7_01_v3 530554
annotation at Pf3D7_02_v3 503316
annota

...

annotation at Pf3D7_14_v3 2625887
annotation at Pf3D7_14_v3 2630597
annotation at Pf3D7_14_v3 2632578
annotation at Pf3D7_14_v3 2635910
annotation at Pf3D7_14_v3 2636395
annotation at Pf3D7_14_v3 2638453
annotation at Pf3D7_14_v3 2638640
annotation at Pf3D7_14_v3 2638643
annotation at Pf3D7_14_v3 2638764
annotation at Pf3D7_14_v3 2639103
annotation at Pf3D7_14_v3 2639159
annotation at Pf3D7_14_v3 2639196
annotation at Pf3D7_14_v3 2639222
annotation at Pf3D7_14_v3 2639315
annotation at Pf3D7_14_v3 2639552
annotation at Pf3D7_14_v3 2639904
annotation at Pf3D7_14_v3 2639935
annotation at Pf3D7_14_v3 2641513
annotation at Pf3D7_14_v3 2643594
annotation at Pf3D7_14_v3 2645558
annotation at Pf3D7_14_v3 2645627
no annotation at  Pf3D7_14_v3 2645627
annotation at Pf3D7_14_v3 2650951
annotation at Pf3D7_14_v3 2653069
annotation at Pf3D7_14_v3 2656219
no annotation at  Pf3D7_14_v3 2656219
annotation at Pf3D7_14_v3 2656353
annotation at Pf3D7_14_v3 2656495
annotation at Pf3D7_14_v3 2657215
no ann

annotation at Pf3D7_14_v3 2845773
annotation at Pf3D7_14_v3 2847899
annotation at Pf3D7_14_v3 2847999
annotation at Pf3D7_14_v3 2848896
annotation at Pf3D7_14_v3 2852502
annotation at Pf3D7_14_v3 2852728
annotation at Pf3D7_14_v3 2852734
annotation at Pf3D7_14_v3 2853115
annotation at Pf3D7_14_v3 2853240
annotation at Pf3D7_14_v3 2853446
annotation at Pf3D7_14_v3 2854734
annotation at Pf3D7_14_v3 2857328
annotation at Pf3D7_14_v3 2857726
annotation at Pf3D7_14_v3 2858480
annotation at Pf3D7_14_v3 2858481
annotation at Pf3D7_14_v3 2858723
annotation at Pf3D7_14_v3 2858940
annotation at Pf3D7_14_v3 2859368
annotation at Pf3D7_14_v3 2859950
annotation at Pf3D7_14_v3 2861876
no annotation at  Pf3D7_14_v3 2861876
annotation at Pf3D7_14_v3 2865152
annotation at Pf3D7_14_v3 2865239
annotation at Pf3D7_14_v3 2865781
no annotation at  Pf3D7_14_v3 2865781
annotation at Pf3D7_14_v3 2865839
no annotation at  Pf3D7_14_v3 2865839
annotation at Pf3D7_14_v3 2868095
annotation at Pf3D7_14_v3 2868356
no

annotation at Pf3D7_14_v3 3009028
annotation at Pf3D7_14_v3 3009347
annotation at Pf3D7_14_v3 3009879
annotation at Pf3D7_14_v3 3012813
annotation at Pf3D7_14_v3 3013005
annotation at Pf3D7_14_v3 3013019
annotation at Pf3D7_14_v3 3013214
annotation at Pf3D7_14_v3 3013280
annotation at Pf3D7_14_v3 3013429
annotation at Pf3D7_14_v3 3013491
annotation at Pf3D7_14_v3 3013575
annotation at Pf3D7_14_v3 3013706
annotation at Pf3D7_14_v3 3014048
annotation at Pf3D7_14_v3 3014069
annotation at Pf3D7_14_v3 3014254
annotation at Pf3D7_14_v3 3014321
annotation at Pf3D7_14_v3 3014805
annotation at Pf3D7_14_v3 3015527
annotation at Pf3D7_14_v3 3015543
annotation at Pf3D7_14_v3 3015639
annotation at Pf3D7_14_v3 3015728
annotation at Pf3D7_14_v3 3015729
annotation at Pf3D7_14_v3 3015740
annotation at Pf3D7_14_v3 3015779
annotation at Pf3D7_14_v3 3016891
annotation at Pf3D7_14_v3 3017013
annotation at Pf3D7_14_v3 3017025
annotation at Pf3D7_14_v3 3017749
annotation at Pf3D7_14_v3 3017859
annotation at 

annotation at Pf3D7_14_v3 3121106
annotation at Pf3D7_14_v3 3121112
annotation at Pf3D7_14_v3 3121124
annotation at Pf3D7_14_v3 3121141
annotation at Pf3D7_14_v3 3121149
annotation at Pf3D7_14_v3 3121152
annotation at Pf3D7_14_v3 3121223
annotation at Pf3D7_14_v3 3121227
annotation at Pf3D7_14_v3 3121290
annotation at Pf3D7_14_v3 3121362
annotation at Pf3D7_14_v3 3121383
annotation at Pf3D7_14_v3 3121404
annotation at Pf3D7_14_v3 3121408
annotation at Pf3D7_14_v3 3121409
annotation at Pf3D7_14_v3 3121415
annotation at Pf3D7_14_v3 3121425
annotation at Pf3D7_14_v3 3121428
annotation at Pf3D7_14_v3 3121436
annotation at Pf3D7_14_v3 3121522
annotation at Pf3D7_14_v3 3121524
annotation at Pf3D7_14_v3 3121550
annotation at Pf3D7_14_v3 3121559
annotation at Pf3D7_14_v3 3121572
annotation at Pf3D7_14_v3 3121601
annotation at Pf3D7_14_v3 3121617
annotation at Pf3D7_14_v3 3121671
annotation at Pf3D7_14_v3 3121676
annotation at Pf3D7_14_v3 3121724
annotation at Pf3D7_14_v3 3121725
annotation at 

annotation at Pf3D7_14_v3 3223835
annotation at Pf3D7_14_v3 3223943
annotation at Pf3D7_14_v3 3224016
annotation at Pf3D7_14_v3 3224257
annotation at Pf3D7_14_v3 3224395
annotation at Pf3D7_14_v3 3224593
annotation at Pf3D7_14_v3 3224614
annotation at Pf3D7_14_v3 3228726
annotation at Pf3D7_14_v3 3229154
annotation at Pf3D7_14_v3 3229203
annotation at Pf3D7_14_v3 3229299
annotation at Pf3D7_14_v3 3232709
annotation at Pf3D7_14_v3 3232835
annotation at Pf3D7_14_v3 3232972
annotation at Pf3D7_14_v3 3233046
annotation at Pf3D7_14_v3 3233227
annotation at Pf3D7_14_v3 3234728
annotation at Pf3D7_14_v3 3235131
annotation at Pf3D7_14_v3 3235252
annotation at Pf3D7_14_v3 3239429
annotation at Pf3D7_14_v3 3239452
annotation at Pf3D7_14_v3 3239506
annotation at Pf3D7_14_v3 3239531
annotation at Pf3D7_14_v3 3239545
annotation at Pf3D7_14_v3 3239888
annotation at Pf3D7_14_v3 3240001
annotation at Pf3D7_14_v3 3240090
annotation at Pf3D7_14_v3 3240165
annotation at Pf3D7_14_v3 3240169
annotation at 

In [5]:
res.T.to_csv(out_path, sep='\t')